In [3]:
from torch import nn
import torch
from torchsummary import summary

<img src = "https://velog.velcdn.com/images%2Feuisuk-chung%2Fpost%2F88ce391b-98dd-48aa-b214-c454c212ef69%2Fimage.png">

* 논문 : https://arxiv.org/pdf/1409.1556

## [1] VGG16 for cifar10

In [24]:
def conv_2_block(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_3_block(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [25]:
class VGG16(nn.Module):
    def __init__(self, num_classes = 10, dropout_p = 0.3):
        '''
        (channel, height, width)
        Input cifar10 size : (3, 32, 32)
        '''
        super().__init__()
        self.dropout_p = dropout_p
        self.backbone = nn.Sequential(
            conv_2_block(3, 64),    # (64, 16, 16)
            conv_2_block(64, 128),  # (128, 8, 8)
            conv_3_block(128, 256), # (256, 4, 4)
            conv_3_block(256, 512), # (512, 2, 2)
            conv_3_block(512, 512)  # (512, 1, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(64, 10)
        )
    
    def forward(self, x):
        out = self.backbone(x)
        # Flatten
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

In [26]:
model = VGG16()
summary(model, (3, 32, 32), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

## [2] VGG19 for ImageNet 

In [19]:
def conv_2_block(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_4_block(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [22]:
class VGG19(nn.Module):
    def __init__(self, num_classes = 10, dropout_p = 0.3):
        '''
        (channel, height, width)
        Input ImageNet size : (3, 224, 224)
        '''
        super().__init__()
        self.dropout_p = dropout_p
        self.backbone = nn.Sequential(
            conv_2_block(3, 64),    # (64, 112, 112)
            conv_2_block(64, 128),  # (128, 56, 56)
            conv_4_block(128, 256), # (256, 28, 28)
            conv_4_block(256, 512), # (512, 14, 14)
            conv_4_block(512, 512)  # (512, 7, 7)
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(64, 10)
        )
    
    def forward(self, x):
        out = self.backbone(x)
        # Flatten
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

In [23]:
model = VGG19()
summary(model, (3, 224, 224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## [3] A, B, C모형구현

In [6]:
def conv_1(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_2(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

def conv_3_1(in_channels, out_channels):
    model = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size = 1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [9]:
class VGG_model(nn.Module):
    def __init__(self, model_type, dropout_p, num_classes):
        super().__init__()
        if model_type == 'A':
            self.backbone = nn.Sequential(
                conv_1(3,64),
                conv_1(64,128),
                conv_2(128,256),
                conv_2(256,512),
                conv_2(512, 512)
            )
        elif model_type == 'B':
            self.backbone = nn.Sequential(
                conv_2(3,64),
                conv_2(64,128),
                conv_2(128,256),
                conv_2(256,512),
                conv_2(512, 512)
            )
        elif model_type == 'C':
            self.backbone = nn.Sequential(
                conv_2(3,64),
                conv_2(64,128),
                conv_3_1(128,256),
                conv_3_1(256,512),
                conv_3_1(512, 512)
            )
            
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7,4096),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(4096,num_classes)
        )
        
    def forward(self, x):
        out = self.backbone(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [10]:
model = VGG_model('A', .3, 1000)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [11]:
model = VGG_model('B', .3, 1000)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [12]:
model = VGG_model('C', .3, 1000)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## [4] 요약

In [1]:
import torch
from torch import nn
from torchsummary import summary

In [2]:
def conv_block(num_layer, in_channels, out_channels):
    if num_layer == '1':
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        )
    if num_layer == '2':
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        )
    elif num_layer == '3_1':
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        )
    elif num_layer == '3':
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        )
    elif num_layer == '4':
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        )
    return block

In [3]:
class VGG_blueprint(nn.Module):
    '''
    ### Linear입력 ###
    32 X 32 : 512 * 1 * 1 -> fc_features = 1
    224 X 224 : 512 * 7 * 7 -> fc_features = 7
    512 X 512 : 512 * 32 * 32 -> fc_features = 32
    '''
    def __init__(self, model_type, dropout_p, fc_features, num_classes):
        super().__init__()
        self.model_type = model_type
        self.dropout_p = dropout_p

        if model_type == 'A':
            self.backbone = nn.Sequential(
                conv_block('1',3,64),
                conv_block('1',64,128),
                conv_block('2',128,256),
                conv_block('2',256,512),
                conv_block('2',512,512)
            )
        elif model_type == 'B':
            self.backbone = nn.Sequential(
                conv_block('2',3,64),
                conv_block('2',64,128),
                conv_block('2',128,256),
                conv_block('2',256,512),
                conv_block('2',512,512)
            )
        elif model_type == 'C':
            self.backbone = nn.Sequential(
                conv_block('2',3,64),
                conv_block('2',64,128),
                conv_block('3_1',128,256),
                conv_block('3_1',256,512),
                conv_block('3_1',512,512)
            )
        elif model_type == 'D':
            self.backbone = nn.Sequential(
                conv_block('2',3,64),
                conv_block('2',64,128),
                conv_block('3',128,256),
                conv_block('3',256,512),
                conv_block('3',512,512)
            )
        elif model_type == 'E':
            self.backbone = nn.Sequential(
                conv_block('2',3,64),
                conv_block('2',64,128),
                conv_block('4',128,256),
                conv_block('4',256,512),
                conv_block('4',512,512)
            )
        else:
            raise Exception('Please select in A,B,C,D,E !!!')
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * fc_features * fc_features, 4096),
            nn.ReLU(),
            nn.Dropout(self.dropout_p),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(self.dropout_p),
            nn.Linear(4096,num_classes)
        )
        self._init_layer()
        
    def _init_layer(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        out = self.backbone(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [4]:
model = VGG_blueprint('A', .3, 7, 10)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [10]:
model = VGG_blueprint('B', .3, 7, 10)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [11]:
model = VGG_blueprint('C', .3, 7, 10)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [12]:
model = VGG_blueprint('D', .3, 7, 10)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [6]:
model = VGG_blueprint('E', .3, 7, 1000)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [5]:
from torchvision.models import vgg19
model = vgg19()
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## cifar10 사용 예시

In [ ]:
def training(self, model, data_loader, criterion, optimizer, device, batch_size):
    model.train()
    train_loss = 0.
    train_acc = 0.
    with tqdm(data_loader, unit = 'batch') as tepoch:
        for i, (X, y) in enumerate(tepoch):
            tepoch.set_description('Training')
            X = X.to(device)
            y = y.to(device)
            
            y_hat = model(X)
            loss = criterion(y_hat, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            iter_loss = loss.item()
            train_loss += iter_loss
            
            pred = y_hat.max(1, keepdim = True)[1]
            iter_acc = pred.eq(y.data.view_as(pred)).sum().item()
            train_acc += iter_acc
            
            tepoch.set_postfix(iteration_num = f'[{i}/{len(data_loader)}]',
                              train_iter_loss = f'{iter_loss / batch_size:.3f}',
                              train_iter_accuracy = f'{iter_acc / batch_size*100:.2f}%')
            
    train_loss = train_loss / len(data_loader)
    train_acc = train_acc / len(data_loader)
    return train_loss, train_acc

def evaluation(self, model, data_loader, criterion, device, batch_size):
    model.eval()
    valid_loss = 0.
    valid_acc = 0.
    with torch.no_grad():
        with tqdm(data_loader, unit = 'batch') as tepoch:
            for i, (X, y) in enumerate(tepoch):
                tepoch.set_description('Evaluation')
                X = X.to(device)
                y = y.to(device)
                
                y_hat = model(X)
                loss = criterion(y_hat, y)
                
                iter_loss = loss.item()
                valid_loss += iter_loss
                
                pred = y_hat.max(1, keepdim = True)[1]
                iter_acc = pred.eq(y.data.view_as(pred)).sum().item()
                valid_acc += iter_acc
                
                tepoch.set_postfix(iteration_num = f'[{i}/{len(data_loader)}]',
                                  valid_iter_loss = f'{iter_loss / batch_size:.3f}',
                                  valid_iter_accuracy = f'{iter_acc / batch_size*100:.2f}%')
                
    valid_loss = valid_loss / len(data_loader)
    valid_acc = valid_acc / len(data_loader)
    return valid_loss, valid_acc